In [150]:
from PIL import Image
import numpy as np
import os

In [151]:
DIRNAME = 'smileys_dataset_all/'
SIZE = 32
files = os.listdir(DIRNAME)
N_data = len(files) * 2
train_data = np.zeros((N_data, SIZE, SIZE, 3))
for i,file in enumerate(files):
    im = Image.open(DIRNAME + file).convert('RGB').resize((SIZE, SIZE))
    train_data[i] = np.array(im).astype(np.float32)
    train_data[i+len(files)] = train_data[i][:,::-1,:]

C:\Users\suzuk\Anaconda3\lib\site-packages\PIL\Image.py:888: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [152]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

In [153]:
N_noise = 100
def generator_model():
    model = Sequential()
    model.add(Dense(1024, input_dim = N_noise))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(128 * 8 * 8))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((128, 8, 8)))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, kernel_size = 5, strides = 1, padding = 'same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(3, kernel_size = 5, strides = 1, padding = 'same'))
    model.add(Activation('tanh'))
    return model

In [154]:
def discriminator_model():
    model = Sequential()
    model.add(Conv2D(64, kernel_size = 5, strides = 2, padding = 'same', input_shape = (3, 32, 32)))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(128, kernel_size = 5, strides = 2))
    model.add(LeakyReLU(0.2))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [155]:
BATCH_SIZE = 16
NUM_EPOCH = 10000
GENERATED_IMAGE_PATH = 'generated_images/'
if not os.path.exists(GENERATED_IMAGE_PATH):
    os.mkdir(GENERATED_IMAGE_PATH)

def train():
    X_train = (train_data - 127.5) / 127.5
    X_train = X_train.reshape(N_data, 3, SIZE, SIZE)
    discriminator = discriminator_model()
    opt_d = Adam(lr = 1e-5, beta_1 = 0.1)
    discriminator.compile(loss='binary_crossentropy', optimizer=opt_d)
    
    discriminator.trainable = False
    generator = generator_model()
    dcgan = Sequential([generator, discriminator])
    opt_g = Adam(lr = 2e-5, beta_1 = 0.5)    
    dcgan.compile(loss='binary_crossentropy', optimizer=opt_g)
    
    num_batches = int(N_data / BATCH_SIZE)
    for epoch in range(NUM_EPOCH):
        
        for index in range(num_batches):
            noise = np.array([np.random.uniform(-1, 1, N_noise) for _ in range(BATCH_SIZE)])
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = generator.predict(noise, verbose=0)
        
            if index == num_batches - 1 and epoch % 100 == 0:
                for n,image in enumerate(generated_images):
                #image = combine_images(generated_images)
                    image = image*127.5 + 127.5
                    Image.fromarray(image.astype(np.uint8).reshape(SIZE,SIZE,3)) \
                                                      .save(GENERATED_IMAGE_PATH+"%03d_%03d.png" % (epoch, n))
                
            X = np.concatenate((image_batch, generated_images))
            y = [1]*BATCH_SIZE + [0]*BATCH_SIZE
            d_loss = discriminator.train_on_batch(X, y)
            
            noise = np.array([np.random.uniform(-1, 1, N_noise) for _ in range(BATCH_SIZE)])
            g_loss = dcgan.train_on_batch(noise, [1]*BATCH_SIZE)
            if index  == num_batches - 1:
                print("epoch: %d, batch: %d, g_loss: %f, d_loss: %f" % (epoch, index, g_loss, d_loss))
    generator.save_weights('generator.h5')
    discriminator.save_weights('discriminator.h5')

In [ ]:
train()

C:\Users\suzuk\Anaconda3\lib\site-packages\keras\engine\training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 0, batch: 158, g_loss: 1.194334, d_loss: 0.220696
epoch: 1, batch: 158, g_loss: 1.343699, d_loss: 0.134516
epoch: 2, batch: 158, g_loss: 1.807332, d_loss: 0.092407
epoch: 3, batch: 158, g_loss: 2.342063, d_loss: 0.056937
epoch: 4, batch: 158, g_loss: 2.854997, d_loss: 0.028085
epoch: 5, batch: 158, g_loss: 3.378094, d_loss: 0.025223
epoch: 6, batch: 158, g_loss: 2.977746, d_loss: 0.019957
epoch: 7, batch: 158, g_loss: 1.234505, d_loss: 0.054137
epoch: 8, batch: 158, g_loss: 0.939212, d_loss: 0.090269
epoch: 9, batch: 158, g_loss: 1.173285, d_loss: 0.224892
epoch: 10, batch: 158, g_loss: 1.580491, d_loss: 0.207268
epoch: 11, batch: 158, g_loss: 1.878146, d_loss: 0.156171
epoch: 12, batch: 158, g_loss: 1.773909, d_loss: 0.209737
epoch: 13, batch: 158, g_loss: 1.717584, d_loss: 0.229181
epoch: 14, batch: 158, g_loss: 1.833476, d_loss: 0.216702
epoch: 15, batch: 158, g_loss: 1.496975, d_loss: 0.245243
epoch: 16, batch: 158, g_loss: 1.712417, d_loss: 0.303673
epoch: 17, batch: 158, g

epoch: 141, batch: 158, g_loss: 0.772512, d_loss: 0.620561
epoch: 142, batch: 158, g_loss: 0.850958, d_loss: 0.692481
epoch: 143, batch: 158, g_loss: 0.801662, d_loss: 0.721771
epoch: 144, batch: 158, g_loss: 0.784031, d_loss: 0.681772
epoch: 145, batch: 158, g_loss: 0.952047, d_loss: 0.627033
epoch: 146, batch: 158, g_loss: 0.764674, d_loss: 0.652340
epoch: 147, batch: 158, g_loss: 0.821072, d_loss: 0.687809
epoch: 148, batch: 158, g_loss: 0.717226, d_loss: 0.652959
epoch: 149, batch: 158, g_loss: 0.835728, d_loss: 0.649216
epoch: 150, batch: 158, g_loss: 0.892598, d_loss: 0.687645
epoch: 151, batch: 158, g_loss: 0.720925, d_loss: 0.635814
epoch: 152, batch: 158, g_loss: 0.793316, d_loss: 0.619117
epoch: 153, batch: 158, g_loss: 0.815546, d_loss: 0.630795
epoch: 154, batch: 158, g_loss: 0.805020, d_loss: 0.662074
epoch: 155, batch: 158, g_loss: 0.744910, d_loss: 0.679368
epoch: 156, batch: 158, g_loss: 0.787130, d_loss: 0.640593
epoch: 157, batch: 158, g_loss: 0.761006, d_loss: 0.6233

epoch: 280, batch: 158, g_loss: 0.952636, d_loss: 0.586925
epoch: 281, batch: 158, g_loss: 0.950625, d_loss: 0.576347
epoch: 282, batch: 158, g_loss: 0.938780, d_loss: 0.571096
epoch: 283, batch: 158, g_loss: 1.109975, d_loss: 0.588323
epoch: 284, batch: 158, g_loss: 0.949363, d_loss: 0.584662
epoch: 285, batch: 158, g_loss: 0.990461, d_loss: 0.586891
epoch: 286, batch: 158, g_loss: 0.917811, d_loss: 0.574030
epoch: 287, batch: 158, g_loss: 0.869311, d_loss: 0.586591
epoch: 288, batch: 158, g_loss: 0.874662, d_loss: 0.599607
epoch: 289, batch: 158, g_loss: 0.875694, d_loss: 0.681184
epoch: 290, batch: 158, g_loss: 0.789043, d_loss: 0.657277
epoch: 291, batch: 158, g_loss: 0.943442, d_loss: 0.623281
epoch: 292, batch: 158, g_loss: 0.877934, d_loss: 0.693952
epoch: 293, batch: 158, g_loss: 0.905672, d_loss: 0.661452
epoch: 294, batch: 158, g_loss: 0.925424, d_loss: 0.679770
epoch: 295, batch: 158, g_loss: 1.061377, d_loss: 0.607646
epoch: 296, batch: 158, g_loss: 0.952818, d_loss: 0.6186

epoch: 419, batch: 158, g_loss: 1.020479, d_loss: 0.558198
epoch: 420, batch: 158, g_loss: 1.009816, d_loss: 0.542535
epoch: 421, batch: 158, g_loss: 1.134499, d_loss: 0.591733
epoch: 422, batch: 158, g_loss: 1.128282, d_loss: 0.581690
epoch: 423, batch: 158, g_loss: 1.076189, d_loss: 0.591587
epoch: 424, batch: 158, g_loss: 1.103813, d_loss: 0.608528
epoch: 425, batch: 158, g_loss: 1.046550, d_loss: 0.556078
epoch: 426, batch: 158, g_loss: 0.973815, d_loss: 0.524168
epoch: 427, batch: 158, g_loss: 1.025541, d_loss: 0.589930
epoch: 428, batch: 158, g_loss: 0.903458, d_loss: 0.690157
epoch: 429, batch: 158, g_loss: 1.109212, d_loss: 0.568990
epoch: 430, batch: 158, g_loss: 0.966880, d_loss: 0.587616
epoch: 431, batch: 158, g_loss: 1.076471, d_loss: 0.613190
epoch: 432, batch: 158, g_loss: 0.879539, d_loss: 0.592782
epoch: 433, batch: 158, g_loss: 1.113900, d_loss: 0.520138
epoch: 434, batch: 158, g_loss: 0.950534, d_loss: 0.554556
epoch: 435, batch: 158, g_loss: 0.897019, d_loss: 0.6039

...

epoch: 7688, batch: 158, g_loss: 1.839236, d_loss: 0.088006
epoch: 7689, batch: 158, g_loss: 3.965748, d_loss: 0.084201
epoch: 7690, batch: 158, g_loss: 3.719355, d_loss: 0.055702
epoch: 7691, batch: 158, g_loss: 2.780918, d_loss: 0.022761
epoch: 7692, batch: 158, g_loss: 2.788686, d_loss: 0.037636
epoch: 7693, batch: 158, g_loss: 1.591389, d_loss: 0.036291
epoch: 7694, batch: 158, g_loss: 3.447321, d_loss: 0.018248
epoch: 7695, batch: 158, g_loss: 3.597474, d_loss: 0.008332
epoch: 7696, batch: 158, g_loss: 3.179665, d_loss: 0.039340
epoch: 7697, batch: 158, g_loss: 4.566286, d_loss: 0.033888
epoch: 7698, batch: 158, g_loss: 3.168576, d_loss: 0.034048
epoch: 7699, batch: 158, g_loss: 3.485370, d_loss: 0.125934
epoch: 7700, batch: 158, g_loss: 3.459565, d_loss: 0.021628
epoch: 7701, batch: 158, g_loss: 3.133804, d_loss: 0.016810
epoch: 7702, batch: 158, g_loss: 2.704705, d_loss: 0.033967
epoch: 7703, batch: 158, g_loss: 4.289076, d_loss: 0.010264
epoch: 7704, batch: 158, g_loss: 2.61712

epoch: 7825, batch: 158, g_loss: 3.251922, d_loss: 0.021321
epoch: 7826, batch: 158, g_loss: 4.774542, d_loss: 0.009273
epoch: 7827, batch: 158, g_loss: 7.394534, d_loss: 0.059455
epoch: 7828, batch: 158, g_loss: 4.799938, d_loss: 0.025925
epoch: 7829, batch: 158, g_loss: 3.778708, d_loss: 0.048992
epoch: 7830, batch: 158, g_loss: 4.441224, d_loss: 0.011553
epoch: 7831, batch: 158, g_loss: 5.000230, d_loss: 0.014653
epoch: 7832, batch: 158, g_loss: 5.245280, d_loss: 0.007882
epoch: 7833, batch: 158, g_loss: 3.724530, d_loss: 0.020655
epoch: 7834, batch: 158, g_loss: 4.427366, d_loss: 0.030268
epoch: 7835, batch: 158, g_loss: 2.911304, d_loss: 0.063953
epoch: 7836, batch: 158, g_loss: 2.546819, d_loss: 0.040913
epoch: 7837, batch: 158, g_loss: 1.952165, d_loss: 0.082018
epoch: 7838, batch: 158, g_loss: 2.132276, d_loss: 0.030207
epoch: 7839, batch: 158, g_loss: 3.251269, d_loss: 0.036441
epoch: 7840, batch: 158, g_loss: 2.560333, d_loss: 0.081866
epoch: 7841, batch: 158, g_loss: 5.05598

epoch: 7962, batch: 158, g_loss: 3.810290, d_loss: 0.028062
epoch: 7963, batch: 158, g_loss: 4.261729, d_loss: 0.031794
epoch: 7964, batch: 158, g_loss: 3.845792, d_loss: 0.004634
epoch: 7965, batch: 158, g_loss: 3.634183, d_loss: 0.001245
epoch: 7966, batch: 158, g_loss: 3.961684, d_loss: 0.015519
epoch: 7967, batch: 158, g_loss: 4.581284, d_loss: 0.030503
epoch: 7968, batch: 158, g_loss: 3.595261, d_loss: 0.004202
epoch: 7969, batch: 158, g_loss: 4.092791, d_loss: 0.011647
epoch: 7970, batch: 158, g_loss: 3.530231, d_loss: 0.035533
epoch: 7971, batch: 158, g_loss: 4.369126, d_loss: 0.021688
epoch: 7972, batch: 158, g_loss: 5.333268, d_loss: 0.013648
epoch: 7973, batch: 158, g_loss: 2.050385, d_loss: 0.061305
epoch: 7974, batch: 158, g_loss: 2.460670, d_loss: 0.014209
epoch: 7975, batch: 158, g_loss: 1.431784, d_loss: 0.122069
epoch: 7976, batch: 158, g_loss: 3.489813, d_loss: 0.011077
epoch: 7977, batch: 158, g_loss: 3.115600, d_loss: 0.005985
epoch: 7978, batch: 158, g_loss: 2.71858

epoch: 8099, batch: 158, g_loss: 3.192922, d_loss: 0.006365
epoch: 8100, batch: 158, g_loss: 2.027593, d_loss: 0.089494
epoch: 8101, batch: 158, g_loss: 3.735604, d_loss: 0.008500
epoch: 8102, batch: 158, g_loss: 3.450912, d_loss: 0.169592
epoch: 8103, batch: 158, g_loss: 1.953830, d_loss: 0.051233
epoch: 8104, batch: 158, g_loss: 2.320124, d_loss: 0.015269
epoch: 8105, batch: 158, g_loss: 2.093086, d_loss: 0.009614
epoch: 8106, batch: 158, g_loss: 2.974447, d_loss: 0.046665
epoch: 8107, batch: 158, g_loss: 2.126081, d_loss: 0.048163
epoch: 8108, batch: 158, g_loss: 3.370529, d_loss: 0.047082
epoch: 8109, batch: 158, g_loss: 2.349442, d_loss: 0.158093
epoch: 8110, batch: 158, g_loss: 2.680967, d_loss: 0.127612
epoch: 8111, batch: 158, g_loss: 2.965838, d_loss: 0.100716
epoch: 8112, batch: 158, g_loss: 1.518279, d_loss: 0.090091
epoch: 8113, batch: 158, g_loss: 4.233352, d_loss: 0.009161
epoch: 8114, batch: 158, g_loss: 4.399081, d_loss: 0.029596
epoch: 8115, batch: 158, g_loss: 2.75082

epoch: 8236, batch: 158, g_loss: 3.298527, d_loss: 0.006364
epoch: 8237, batch: 158, g_loss: 4.256935, d_loss: 0.034890
epoch: 8238, batch: 158, g_loss: 3.172721, d_loss: 0.014237
epoch: 8239, batch: 158, g_loss: 1.211155, d_loss: 0.146605
epoch: 8240, batch: 158, g_loss: 3.647803, d_loss: 0.007396
epoch: 8241, batch: 158, g_loss: 2.336844, d_loss: 0.031547
epoch: 8242, batch: 158, g_loss: 2.718775, d_loss: 0.006071
epoch: 8243, batch: 158, g_loss: 3.062989, d_loss: 0.020265
epoch: 8244, batch: 158, g_loss: 4.095866, d_loss: 0.007500
epoch: 8245, batch: 158, g_loss: 3.124026, d_loss: 0.028872
epoch: 8246, batch: 158, g_loss: 3.341980, d_loss: 0.016216
epoch: 8247, batch: 158, g_loss: 3.718011, d_loss: 0.022744
epoch: 8248, batch: 158, g_loss: 4.518068, d_loss: 0.026966
epoch: 8249, batch: 158, g_loss: 3.483123, d_loss: 0.027917
epoch: 8250, batch: 158, g_loss: 2.875586, d_loss: 0.006380
epoch: 8251, batch: 158, g_loss: 2.153050, d_loss: 0.031944
epoch: 8252, batch: 158, g_loss: 1.08334